# ZBIW Data Librarian Modul 2 - Präsenztag

## Umlaut-Problematik

Allgemeine Infos: https://docs.python.org/3/howto/unicode.html

Bytes als String decodieren: https://docs.python.org/3/library/stdtypes.html#bytes.decode

Strings als Bytes codieren: https://docs.python.org/3/library/stdtypes.html#str.encode

Verfügbare Encodings: https://docs.python.org/3/library/codecs.html#standard-encodings

Codecs: https://docs.python.org/3/library/codecs.html#module-codecs

### Dateien lesen und schreiben mit korrektem Encoding

Python nimmt implizit das Encoding des Systems an.

In [ ]:
import locale
locale.getpreferredencoding() 

Um explizit das Encoding zu setzen, wird der Parameter `encoding` der `open` Funktion verwendet:

In [ ]:
with open('data/test.txt', 'r', encoding='cp1252') as f:
    for line in f:
        print(line)

### Urllib

Urllib kann nicht mit Umlauten oder anderen speziellen Sonderzeichen in der URL umgehen, daher muss bei Vorhandensein die URL vorher umgewandelt werden. Das geht beispielsweise mit der Funktion `urllib.parse.quote`.

Link zur Doku: https://docs.python.org/3/library/urllib.parse.html#url-quoting

In [ ]:
import json
import urllib.request
import urllib.parse

# Umlaute in der URL:
url = "https://www.bibsonomy.org/json/search/" + urllib.parse.quote("Lösungen") + "?items=1000&duplicates=merged"

f = urllib.request.urlopen(url)
print(type(f))

In [ ]:
# Prüfen wie die geparste URL aussieht:
url

In [ ]:
read = f.read() # read() ist Methode des HTTPResponse-Objektes
print(type(read)) # Rückgabe-Typ = bytes

Wenn das Ergebnis vom Request empfangen wird, wird der `decode` Funktion explizit das Encoding als Parameter mitgegeben, um korrekt encodete Daten zu erhalten.

Die `decode` Funktion ist Teil der Standard-Library: https://docs.python.org/3/library/stdtypes.html#bytes.decode

In [ ]:
result = read.decode('utf-8') # Decoden der Bytes in einen str

# Ergebnis ist str, muss erst noch als JSON verarbeitet werden (Ergebnis = dict)
data = json.loads(result) # json.loads() lädt JSON Daten aus einem String
data

### Requests

https://2.python-requests.org/en/master/user/quickstart/#response-content

"When you make a request, Requests makes educated guesses about the encoding of the response based on the HTTP headers. The text encoding guessed by Requests is used when you access `r.text`. You can find out what encoding Requests is using, and change it, using the `r.encoding` property:"

In [ ]:
# requests hat kein Problem mit Umlauten in URLs
url = "https://www.bibsonomy.org/json/search/Lösungen?items=1000&duplicates=merged"

In [ ]:
import requests

result = requests.get(url) # result ist ein requests.models.Response Objekt
result.encoding # Das Encoding der Response lässt sich abfragen

In [ ]:
# Encoding lässt sich auch ändern:
result.encoding = 'ISO-8859-1'

In [ ]:
data = result.json()
data # die JSON Daten sind jetzt im vorgegebenen Encoding

---

## REST-APIs abfragen

Weiteres Beispiel CrossRef: https://github.com/CrossRef/rest-api-doc

In [ ]:
url = "https://api.crossref.org/works?query.author=herpers&rows=100"
result = requests.get(url)
data = result.json()

In [ ]:
data.keys() # was für Keys haben wir

In [ ]:
data['message']['total-results'] # wie viele Ergebnisse laut der Antwort der API

In [ ]:
data['message'].keys() # was für Keys haben wir innerhalb des Dictionaries 'message'

In [ ]:
type(data['message']['items']) # welche Datenstruktur 

In [ ]:
items = data['message']['items']
len(items) # wie viele Ergebnisse haben wir bekommen

In [ ]:
items[0] # Einblick ins Ergebnis

Einige APIs bieten **Filter** als URL-Parameter an, um die Ergebnisse schon bei Abfrage nach bestimmten Kriterien zu filtern. Es gibt eine ganze Menge verfügbarer Filter bei Crossref, siehe Doku: https://github.com/CrossRef/rest-api-doc#filter-names

Hier ein Beispiel, mit dem die Ergebnisliste nach Jahreszahl des Publikationsjahres gefiltert wird:

In [ ]:
url = "https://api.crossref.org/works?query.author=herpers&rows=100&filter=from-pub-date:2010,until-pub-date:2015"
result = requests.get(url)
data = result.json()
data['message']['total-results']

### Kombinierte Abfragen

Beispiel [SemanticScholar](https://api.semanticscholar.org/): Hier kann man Daten zu Papers abfragen, wenn man denn den Identifier kennt. Hier ist ein Use-Case vorstellbar, in dem man bereits eine Liste von Identifiern hat, und nun jeweils die Daten abfragen möchte. Da bietet sich Iteration an:

In [ ]:
base_url = "https://api.semanticscholar.org/v1/paper/"

# die Liste der Identifier kann aus einer Quelle stammen, z.B. von einer Datei eingelesen werden, die man hierfür pflegt
identifiers = ["arXiv:1705.10311", "arXiv:1705.10312", "arXiv:1705.10313", "arXiv:1705.10314"]

results = []

for identifier in identifiers:
    url = base_url + identifier
    result = requests.get(url)
    data = result.json()
    results.append(data)

In [ ]:
len(results)

In [ ]:
results[0].keys()

### Etiquette

Generell ist davon abzuraten, einen fremden Webserver mit allzu häufigen Anfragen zu bombardieren. Vermeidungsstrategien sind hierbei, wie auch in der Crossref-Etiquette angegeben, gezielte Pausen zwischen Abfragen, sowie Caching. Caching ist insbesondere bei wiederholt gleichen Anfragen anzuraten. Bei verschiedenen Anfragen sollte zwischen jeder Anfrage eine kleine Pause eingehalten werden. Hierfür gibt es keine festen Vorschriften, einige Sekunden bis maximal eine Minute sollte idR ausreichend sein.

Um das programmatisch zu erreichen, kann man in Python die sleep Funktion benutzen:

```
import time

for query in queries:
    # Abfrage mit der query an den Webserver
    time.sleep(1000) # 1000ms = 1s warten, bevor die nächste Anweisung ausgeführt wird
```

Tutorial: https://realpython.com/python-sleep/

---

## OAI-PMH APIs mit Python abfragen

Für den Fall, dass das mal für jemanden relevant wird, hier ein Beispiel, wie man eine OAI-PMH API mittels Python abfragen kann.

In diesem Beispiel erfolgt das mit der *pyoai* Library, die eine Drittanbieter-Library ist und daher erst installiert werden muss. Nach Bedarf die folgenden Zeilen hierfür auskommentieren.

In [ ]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c auto pyoai
#!{sys.executable} -m pip install pyoai

In [ ]:
OAI_PHM_URL = 'https://www.ssoar.info/OAIHandler/request'

In [ ]:
from oaipmh.client import Client
from oaipmh.metadata import MetadataRegistry, oai_dc_reader

registry = MetadataRegistry()
registry.registerReader('oai_dc', oai_dc_reader)
client = Client(OAI_PHM_URL, registry)

In [ ]:
# Get single record
record = client.getRecord(metadataPrefix='oai_dc', identifier='oai:gesis.izsoz.de:document/679')
header = record[0]
print('id: {}'.format(header.identifier()))
print('element: {}'.format(header.element()))
print('datestamp: {}'.format(header.datestamp()))

In [ ]:
metadata = record[1].getMap()
if 'creator' in metadata:
    print('creator: {}'.format(metadata['creator']))

In [ ]:
# List records
for record in client.listRecords(metadataPrefix='oai_dc'):
    header = record[0]
    print('id: {}'.format(header.identifier()))
    metadata = record[1].getMap()
    if 'creator' in metadata:
        print('creator: {}'.format(metadata['creator']))
    if 'title' in metadata:
        print('title: {}'.format(metadata['title']))
    print()

---

## XML Verarbeitung

Es gibt viele Libraries, mit denen man in Python XML verarbeiten kann. Eine Liste:

* xml.dom - The Document Object Model API ([Standard](https://docs.python.org/3.5/library/xml.dom.html#)) 
* xml.etree - The ElementTree XML API ([Standard](https://docs.python.org/3.8/library/xml.etree.elementtree.html#))              
* lxml ([Extern](https://lxml.de/index.html))
* BeautifulSoup ([Extern](https://www.crummy.com/software/BeautifulSoup/bs4/doc/))

Im Folgenden ein kleines Beispiel zum Parsen von XML mit den Möglichkeiten der Standard-Library ElementTree.

Doku: https://docs.python.org/3.8/library/xml.etree.elementtree.html#

In [ ]:
url = "https://www.bibsonomy.org/layout/dblp/search/Bibliothek?items=100"

import requests
result = requests.get(url)

type(result)

In [ ]:
result.text

In [ ]:
from xml.etree import ElementTree as ET
from xml.etree.ElementTree import fromstring

xmlString = fromstring(result.text) # Einlesen des Strings als XML

tree = ET.ElementTree(xmlString) # Parsen des XML als Baumstruktur (Element Tree)
root = tree.getroot() # Wurzel des Baums abfragen
root

Wie man sieht, ist `root` vom Typ `Element`, den wir in der Musterlösung zu Aufgabe 1 schon gesehen haben.

In [ ]:
for book in root.findall('book'): # Kindelemente des Wurzelknotens mit bestimmtem Tag suchen
    key = book.get('key') # Attribut des Tags abfragen
    print("Key: " + key)
    titleTag = book.find('title') # weitere Kindknoten suchen
    title = titleTag.text # Text extrahieren
    print("Title: " + title)
    print()